# Partie 2 - Chapitre 3 : Les Trois Architectures d'Agents

## Introduction

Maintenant que nous comprenons les outils (chapitre 0) et LangGraph (chapitre 1), plongeons dans le monde **agentique** !

### Qu'est-ce qu'un Agent ?

Un agent est un système qui :
- **Raisonne** sur une tâche
- **Décide** quels outils utiliser
- **Exécute** des actions
- **Adapte** son comportement selon les résultats

### Les Trois Architectures

Dans ce chapitre, nous allons explorer **3 architectures d'agents** différentes, chacune avec ses forces et cas d'usage :

| Architecture | Principe | Cas d'usage |
|--------------|----------|-------------|
| **SimpleToolAgent** | Orchestration directe d'outils | Tâches simples, efficacité |
| **SimpleReActAgent** | Raisonnement explicite (Thought→Action→Observation) | Compréhension du processus, débogage |
| **HierarchicalPlanner** | Décomposition en sous-tâches avec dépendances | Tâches complexes multi-étapes |

### Notre Cas d'Usage : VéloCorp

Nous allons travailler avec les données de **VéloCorp**, une entreprise de vélos électriques. Les données incluent :
- 📚 FAQs (garanties, compatibilité)
- 📖 Manuels utilisateurs (E-City, Pro-Sécurité)
- 📧 Emails clients
- 🎙️ Transcriptions d'appels

**Objectif pédagogique** : Comprendre comment différentes architectures d'agents résolvent les mêmes problèmes de manière différente.

## Configuration Initiale

Commençons par configurer notre environnement avec les outils VéloCorp.

In [ ]:
# Imports nécessaires
from rag_attack import (
    set_config,
    SimpleToolAgent,
    SimpleReActAgent,
    HierarchicalPlanner,
    azure_search_tool,
    report_writer_tool
)
import json

print("✅ Imports réussis")

In [ ]:
# Configuration Azure (VéloCorp data)
config = {}

set_config(config)

print("✅ Configuration VéloCorp initialisée")
print(f"🔍 Azure Search endpoint : {config['search_endpoint'][:50]}...")
print(f"🤖 Modèle LLM : {config['chat_deployment']}")

### Comprendre les Outils Disponibles

Nous avons deux outils principaux :

1. **azure_search_tool** : Recherche dans la base de connaissances VéloCorp
   - FAQs, manuels, emails, transcriptions
   - Paramètres : `query` (recherche), `top` (nombre de résultats)

2. **report_writer_tool** : Génération de rapports professionnels
   - Formats : Markdown, HTML, Text
   - Types : general, technical, business, customer_analysis
   - Paramètres : `title`, `content`, `report_type`, `format`

In [ ]:
# Test rapide des outils
print("🔧 OUTILS DISPONIBLES :")
print("="*70)

# Test azure_search_tool
print("\n1️⃣  azure_search_tool")
print(f"   Nom : {azure_search_tool.name}")
print(f"   Description : {azure_search_tool.description[:100]}...")

# Test report_writer_tool
print("\n2️⃣  report_writer_tool")
print(f"   Nom : {report_writer_tool.name}")
print(f"   Description : {report_writer_tool.description[:100]}...")

print("\n" + "="*70)
print("✅ Outils prêts à être utilisés par les agents")

## Questions VéloCorp

Chargeons les questions de test pour VéloCorp. Ces questions sont classées par difficulté et couvrent différents aspects de l'entreprise.

In [ ]:
# Charger les questions VéloCorp
with open("tests_tools/sample_data/velocop_questions.json", "r", encoding="utf-8") as f:
    velocop_questions = json.load(f)

print("📋 QUESTIONS VÉLOCORP CHARGÉES :")
print("="*70)

# Questions simples
print("\n🟢 Questions Simples :")
for i, q in enumerate(velocop_questions["questions_simples"][:3], 1):
    print(f"   {i}. {q['question']}")
    print(f"      Outils attendus : {', '.join(q['outils_attendus'])}")

# Questions moyennes
print("\n🟡 Questions Moyennes :")
for i, q in enumerate(velocop_questions["questions_moyennes"][:2], 1):
    print(f"   {i}. {q['question']}")
    print(f"      Outils attendus : {', '.join(q['outils_attendus'])}")

# Questions complexes
print("\n🔴 Questions Complexes :")
for i, q in enumerate(velocop_questions["questions_complexes"][:2], 1):
    print(f"   {i}. {q['question']}")
    print(f"      Outils attendus : {', '.join(q['outils_attendus'])}")

print("\n" + "="*70)

---

# Architecture 1 : SimpleToolAgent

## Le Travailleur Efficace

### Principe

Le **SimpleToolAgent** est l'architecture la plus directe :
1. Le LLM reçoit la question
2. Il décide quels outils appeler
3. Il exécute les outils en séquence
4. Il formule la réponse finale

**Avantages** :
- ✅ Simple et efficace
- ✅ Fiable et prévisible
- ✅ Faible latence
- ✅ Idéal pour la production

**Cas d'usage** :
- Questions directes nécessitant 1-2 outils
- Systèmes de production où la fiabilité est critique
- Applications temps-réel

### Visualisation du Flux

```
Question → LLM → Tool 1 → LLM → Tool 2 → LLM → Réponse
```

### Exemple 1 : Question Simple

In [ ]:
# Créer un SimpleToolAgent
simple_agent = SimpleToolAgent(
    config=config,
    tools=[azure_search_tool],
    system_prompt="Tu es un assistant expert sur VéloCorp. Réponds de manière claire et concise.",
    max_iterations=3,
    verbose=True  # Afficher les étapes d'exécution
)

print("✅ SimpleToolAgent créé")
print(f"🔧 Outils disponibles : {len(simple_agent.tools)}")
print(f"🔄 Iterations max : {simple_agent.max_iterations}")

In [ ]:
# Question simple sur les produits
question_simple = "Quels sont les modèles de vélos électriques proposés par VéloCorp ?"

print("❓ QUESTION :")
print(question_simple)
print("\n" + "="*70 + "\n")

# Invoquer l'agent
response_simple = simple_agent.invoke(question_simple)

print("\n" + "="*70)
print("\n💡 RÉPONSE :")
print(response_simple)

**Observation** : Le SimpleToolAgent a :
1. Reçu la question
2. Décidé d'utiliser `azure_search_tool`
3. Cherché dans la base de connaissances
4. Formulé une réponse claire

**Questions de compréhension** :
1. Combien d'itérations l'agent a-t-il utilisées ?
2. Quels mots-clés a-t-il probablement cherchés ?
3. Pourquoi est-ce suffisant pour cette question ?

### Exemple 2 : Multi-outils

Voyons maintenant comment le SimpleToolAgent gère plusieurs outils.

In [ ]:
# Créer un agent avec deux outils
multi_tool_agent = SimpleToolAgent(
    config=config,
    tools=[azure_search_tool, report_writer_tool],
    system_prompt="Tu es un assistant expert sur VéloCorp. Utilise les outils pour créer des rapports professionnels.",
    max_iterations=5,
    verbose=True
)

print("✅ Multi-tool agent créé")
print(f"🔧 Outils : {[tool.name for tool in multi_tool_agent.tools]}")

In [ ]:
# Question nécessitant recherche + génération de rapport
question_rapport = """Crée un rapport technique sur le modèle E-City : 
caractéristiques, fonctionnalités, et recommandations d'usage."""

print("❓ QUESTION :")
print(question_rapport)
print("\n" + "="*70 + "\n")

# Invoquer l'agent
response_rapport = multi_tool_agent.invoke(question_rapport)

print("\n" + "="*70)
print("\n💡 RÉPONSE :")
print(response_rapport[:1000] + "..." if len(response_rapport) > 1000 else response_rapport)

**Observation** : L'agent a probablement :
1. Utilisé `azure_search_tool` pour trouver les infos sur E-City
2. Utilisé `report_writer_tool` pour créer un rapport structuré
3. Combiné les résultats pour une réponse professionnelle

**Questions de compréhension** :
1. Dans quel ordre les outils ont-ils été utilisés ?
2. Pourquoi utiliser un outil de rapport plutôt que laisser le LLM formater directement ?
3. Comment l'agent sait-il quand arrêter d'utiliser des outils ?

### 🎛️ Paramètres Modifiables

Expérimentez avec ces paramètres pour voir leur impact :

In [ ]:
# À VOUS : Modifiez ces paramètres et observez les différences

# Paramètre 1: max_iterations
# Que se passe-t-il avec max_iterations=1 vs max_iterations=10 ?

# Paramètre 2: system_prompt
# Essayez différents prompts système :
# - "Réponds de manière ultra-concise en 2 phrases maximum"
# - "Fournis des réponses détaillées et techniques"
# - "Réponds comme un expert commercial VéloCorp"

# Paramètre 3: verbose
# verbose=False pour une sortie propre
# verbose=True pour voir les détails d'exécution

# Testez ici :
experimental_agent = SimpleToolAgent(
    config=config,
    tools=[azure_search_tool],
    system_prompt="MODIFIEZ ICI",  # ← Changez ceci
    max_iterations=3,  # ← Changez ceci
    verbose=True  # ← Changez ceci
)

# Testez avec votre propre question
ma_question = "Quelle est la garantie sur le modèle E-City ?"
ma_reponse = experimental_agent.invoke(ma_question)
print(ma_reponse)

---

# Architecture 2 : SimpleReActAgent

## Le Penseur Transparent

### Principe

Le **SimpleReActAgent** suit le pattern **ReAct** (Reasoning and Acting) :

```
Boucle pour chaque itération :
  1. THOUGHT (Pensée)   : "Je dois chercher les infos sur les freins"
  2. ACTION (Action)    : Utilise azure_search_tool avec query="freins E-City"
  3. OBSERVATION (Obs.) : "J'ai trouvé 3 documents sur les freins"
  → Répéter jusqu'à avoir assez d'informations
  4. ANSWER (Réponse)   : Formuler la réponse finale
```

**Avantages** :
- ✅ Raisonnement explicite et traçable
- ✅ Excellent pour comprendre le processus de décision
- ✅ Utile pour le débogage
- ✅ Pédagogique - on voit "comment" l'agent pense

**Cas d'usage** :
- Systèmes nécessitant de l'explicabilité
- Débogage de comportements d'agents
- Formation et compréhension des agents
- Systèmes d'IA transparents

### Exemple 1 : Raisonnement Étape par Étape

In [ ]:
# Créer un SimpleReActAgent
react_agent = SimpleReActAgent(
    tools=[azure_search_tool],
    max_iterations=5,
    verbose=True  # IMPORTANT : Pour voir le raisonnement
)

print("✅ SimpleReActAgent créé")
print(f"🔧 Outils disponibles : {len(react_agent.tools)}")
print(f"🔄 Iterations max : {react_agent.max_iterations}")
print("\n💡 Mode verbose activé : vous allez voir TOUT le processus de raisonnement\n")

In [ ]:
# Question moyenne nécessitant du raisonnement
question_react = "Quels sont les problèmes courants signalés par les clients concernant les freins ?"

print("❓ QUESTION :")
print(question_react)
print("\n" + "="*70 + "\n")

# Invoquer l'agent ReAct
response_react = react_agent.invoke(question_react)

print("\n" + "="*70)
print("\n✨ RÉPONSE FINALE :")
print(response_react)

**Observation du Pattern ReAct** :

Regardez attentivement la sortie ci-dessus. Vous devriez voir :

```
ITERATION 1/5
─────────────────────────────────────────

💭 THOUGHT:
   Je dois chercher dans la base de données VéloCorp...

🔧 ACTION:
   Tool: azure_search_tool
   Input: problèmes freins clients

👁️  OBSERVATION:
   Used azure_search_tool and found: [résultats]...

ITERATION 2/5 (si nécessaire)
...
```

Ce pattern montre **comment** l'agent arrive à sa conclusion.

### Exemple 2 : Obtenir la Trace Complète de Raisonnement

Le SimpleReActAgent peut retourner une trace structurée de son raisonnement :

In [ ]:
# Créer un agent sans verbose pour avoir une trace propre
react_agent_trace = SimpleReActAgent(
    tools=[azure_search_tool],
    max_iterations=4,
    verbose=False  # Désactiver l'affichage détaillé
)

question_trace = "Quelles sont les caractéristiques du modèle E-City ?"

# Obtenir la trace de raisonnement
trace = react_agent_trace.get_reasoning_trace(question_trace)

print("🔍 TRACE DE RAISONNEMENT STRUCTURÉE")
print("="*70)
print(f"\n❓ Question: {trace['question']}\n")

# Afficher chaque étape de raisonnement
for i, (thought, obs) in enumerate(zip(trace['reasoning'], trace['observations']), 1):
    print(f"\n{'─'*70}")
    print(f"🔄 ITÉRATION {i}")
    print(f"{'─'*70}")
    print(f"\n💭 Pensée:\n{thought}")
    print(f"\n👁️  Observation:\n{obs[:200]}..." if len(obs) > 200 else f"\n👁️  Observation:\n{obs}")

print(f"\n\n{'='*70}")
print("✨ RÉPONSE FINALE :")
print(f"{'='*70}")
print(trace['final_answer'])

**Questions de compréhension** :
1. Combien d'itérations l'agent a-t-il effectuées avant d'avoir assez d'informations ?
2. Comment l'agent décide-t-il qu'il a assez d'informations ?
3. Quelle est la différence principale avec SimpleToolAgent ?

**Réponse à la question 2** : Le SimpleReActAgent arrête quand :
- Il dit explicitement "J'ai assez d'informations"
- OU il a 3+ observations réussies
- OU il atteint max_iterations

### Comparaison : SimpleToolAgent vs SimpleReActAgent

Testons la même question avec les deux architectures :

In [ ]:
# Question de test
question_comparaison = "Quel est le prix du modèle Sport-Elite ?"

print("="*70)
print("🔬 COMPARAISON DES ARCHITECTURES")
print("="*70)
print(f"\n❓ Question: {question_comparaison}\n")

# Test avec SimpleToolAgent
print("\n" + "─"*70)
print("1️⃣  SIMPLETOOLAGENT (Efficace et Direct)")
print("─"*70)

simple_test = SimpleToolAgent(
    config=config,
    tools=[azure_search_tool],
    max_iterations=3,
    verbose=False
)

import time
start_simple = time.time()
response_simple_comp = simple_test.invoke(question_comparaison)
time_simple = time.time() - start_simple

print(f"⏱️  Temps : {time_simple:.2f}s")
print(f"💡 Réponse : {response_simple_comp}")

# Test avec SimpleReActAgent
print("\n" + "─"*70)
print("2️⃣  SIMPLEREACTAGENT (Raisonnement Explicite)")
print("─"*70)

react_test = SimpleReActAgent(
    tools=[azure_search_tool],
    max_iterations=3,
    verbose=False
)

start_react = time.time()
response_react_comp = react_test.invoke(question_comparaison)
time_react = time.time() - start_react

print(f"⏱️  Temps : {time_react:.2f}s")
print(f"💡 Réponse : {response_react_comp}")

# Analyse
print("\n" + "="*70)
print("📊 ANALYSE")
print("="*70)
print(f"\nSimpleToolAgent : {time_simple:.2f}s - Réponse directe")
print(f"SimpleReActAgent : {time_react:.2f}s - Raisonnement explicite")
print(f"\nDifférence : {abs(time_react - time_simple):.2f}s")
print(f"\n💡 SimpleReActAgent est {'plus lent' if time_react > time_simple else 'plus rapide'} car il explicite chaque étape de raisonnement")
print("💡 Mais cette transparence est précieuse pour la compréhension et le débogage !")

### 🎛️ Paramètres Modifiables

In [ ]:
# À VOUS : Expérimentez avec SimpleReActAgent

# Paramètre 1: max_iterations
# Que se passe-t-il avec max_iterations=2 sur une question complexe ?

# Paramètre 2: verbose
# Comparez verbose=True (détaillé) vs verbose=False (propre)

# Essayez différentes questions VéloCorp :
questions_test = [
    "Quels sont les différents modes d'assistance du E-City ?",
    "Comment entretenir la batterie des vélos VéloCorp ?",
    "Quels accessoires sont compatibles avec le modèle Urbain-Confort ?"
]

mon_react_agent = SimpleReActAgent(
    tools=[azure_search_tool],
    max_iterations=4,  # ← Modifiez ceci
    verbose=True  # ← Modifiez ceci
)

# Testez avec une question de votre choix
ma_question_react = questions_test[0]  # ← Changez l'index ou écrivez votre question
ma_reponse_react = mon_react_agent.invoke(ma_question_react)
print(f"\n✨ Réponse finale : {ma_reponse_react}")

---

# Architecture 3 : HierarchicalPlanner

## L'Organisateur Stratégique

### Principe

Le **HierarchicalPlanner** décompose les tâches complexes en sous-tâches avec gestion des dépendances :

```
1. ANALYSE : Comprendre l'objectif global
2. PLANIFICATION : Créer un plan avec des étapes et dépendances
   - Step 1: Chercher les infos X
   - Step 2: Chercher les infos Y (dépend de Step 1)
   - Step 3: Synthétiser (dépend de Step 1 et 2)
3. VALIDATION : Vérifier que le plan est cohérent
4. EXÉCUTION : Exécuter les étapes dans l'ordre correct
5. ÉVALUATION : Vérifier si l'objectif est atteint
6. REPLANIFICATION : Ajuster si nécessaire
7. FINALISATION : Synthétiser les résultats
```

**Avantages** :
- ✅ Gère des tâches multi-étapes complexes
- ✅ Gestion explicite des dépendances
- ✅ Peut s'adapter si une étape échoue
- ✅ Excellent pour des workflows d'affaires

**Cas d'usage** :
- Analyses nécessitant plusieurs sources de données
- Tâches avec dépendances claires (A doit finir avant B)
- Workflows d'affaires complexes
- Planification stratégique

### Exemple 1 : Tâche Complexe Multi-Étapes

In [ ]:
# Créer un HierarchicalPlanner
planner = HierarchicalPlanner(
    tools=[azure_search_tool],
    max_steps=8
)

print("✅ HierarchicalPlanner créé")
print(f"🔧 Outils disponibles : {len(planner.tools)}")
print(f"📊 Steps max : {planner.max_steps}")
print("\n💡 Ce planner va décomposer les tâches complexes automatiquement\n")

In [ ]:
# Objectif complexe nécessitant une planification
objectif_complexe = """Analyse complète pour un rapport au conseil d'administration : 
1. Identifier tous les modèles de vélos et leurs caractéristiques principales
2. Recenser les problèmes clients les plus fréquents
3. Lister les questions fréquentes sur la garantie
4. Synthétiser une vue d'ensemble de la satisfaction client"""

print("🎯 OBJECTIF COMPLEXE :")
print(objectif_complexe)
print("\n" + "="*70 + "\n")

# Exécuter avec le planner
result_planner = planner.execute(objectif_complexe)

print("\n" + "="*70)
print("📋 PLAN CRÉÉ PAR LE PLANNER")
print("="*70)

# Afficher le plan
for step in result_planner['plan']:
    status_emoji = {
        "pending": "⏳",
        "in_progress": "🔄",
        "completed": "✅",
        "failed": "❌"
    }.get(step['status'], "❓")
    
    print(f"\n{status_emoji} Step {step['step_id']}: {step['description']}")
    if step['dependencies']:
        print(f"   Dépendances: {step['dependencies']}")
    if step['tool']:
        print(f"   Outil: {step['tool']}")
    if step['status'] == 'completed' and step['result']:
        print(f"   Résultat: {step['result'][:150]}..." if len(step['result']) > 150 else f"   Résultat: {step['result']}")

In [ ]:
# Afficher le résultat final synthétisé
print("\n" + "="*70)
print("📊 RÉSULTAT FINAL SYNTHÉTISÉ")
print("="*70)
print(result_planner['final_result'])

print("\n" + "="*70)
print("📈 STATISTIQUES D'EXÉCUTION")
print("="*70)
print(f"\nNombre total d'étapes planifiées: {len(result_planner['plan'])}")
print(f"Étapes complétées: {sum(1 for s in result_planner['plan'] if s['status'] == 'completed')}")
print(f"Étapes avec outils: {sum(1 for s in result_planner['plan'] if s['tool'])}")
print(f"Historique d'exécution: {len(result_planner['execution_history'])} actions")

**Observation du HierarchicalPlanner** :

Le planner a :
1. **Analysé** l'objectif complexe
2. **Créé un plan** avec plusieurs étapes
3. **Identifié les dépendances** (quelles étapes doivent finir avant d'autres)
4. **Exécuté dans l'ordre** en respectant les dépendances
5. **Synthétisé** tous les résultats en une réponse finale

**Questions de compréhension** :
1. Combien d'étapes le planner a-t-il créées ?
2. Y a-t-il des dépendances entre les étapes ? Lesquelles ?
3. Comment le planner décide-t-il de l'ordre d'exécution ?
4. Quelle est la différence avec SimpleToolAgent qui ferait plusieurs appels d'outils ?

### 🎛️ Paramètres Modifiables

In [ ]:
# À VOUS : Expérimentez avec HierarchicalPlanner

# Paramètre : max_steps
# Que se passe-t-il avec max_steps=3 sur un objectif complexe ?
# Vs max_steps=15 ?

mon_planner = HierarchicalPlanner(
    tools=[azure_search_tool],
    max_steps=10  # ← Modifiez ceci
)

# Essayez vos propres objectifs complexes
objectifs_test = [
    "Analyse des réclamations clients sur les 3 derniers mois",
    "Étude de marché : comparer VéloCorp avec les concurrents",
    "Plan d'action pour améliorer la satisfaction client"
]

mon_objectif = objectifs_test[0]  # ← Changez l'index ou écrivez votre objectif

print(f"🎯 Objectif: {mon_objectif}\n")
mon_resultat = mon_planner.execute(mon_objectif)

print("\n📋 Plan créé:")
for step in mon_resultat['plan']:
    status = "✅" if step['status'] == 'completed' else "❌"
    print(f"{status} Step {step['step_id']}: {step['description'][:80]}")

print(f"\n\n💡 Résultat final:\n{mon_resultat['final_result'][:500]}...")

---

# Comparaison des Trois Architectures

## Tableau Comparatif

In [ ]:
# Créer un tableau comparatif
import pandas as pd

comparaison = pd.DataFrame({
    "Critère": [
        "Complexité",
        "Transparence",
        "Fiabilité",
        "Vitesse",
        "Gestion dépendances",
        "Cas d'usage idéal",
        "Itérations moyennes"
    ],
    "SimpleToolAgent": [
        "Faible ⭐",
        "Moyenne ⭐⭐",
        "Excellente ⭐⭐⭐",
        "Rapide ⭐⭐⭐",
        "Non",
        "Questions directes, production",
        "1-2"
    ],
    "SimpleReActAgent": [
        "Moyenne ⭐⭐",
        "Haute ⭐⭐⭐",
        "Excellente ⭐⭐⭐",
        "Moyenne ⭐⭐",
        "Non",
        "Débogage, compréhension",
        "2-3"
    ],
    "HierarchicalPlanner": [
        "Haute ⭐⭐⭐",
        "Haute ⭐⭐⭐",
        "Excellente ⭐⭐⭐",
        "Plus lent ⭐",
        "Oui ✅",
        "Analyses complexes, workflows",
        "3-5 steps"
    ]
})

print("\n📊 COMPARAISON DES TROIS ARCHITECTURES")
print("="*80)
print(comparaison.to_string(index=False))
print("="*80)

## Test Comparatif sur la Même Question

In [ ]:
# Question de test pour les 3 architectures
question_benchmark = "Quelles sont les caractéristiques du modèle E-City et les problèmes courants rapportés par les clients ?"

print("🔬 TEST COMPARATIF")
print("="*80)
print(f"\n❓ Question: {question_benchmark}\n")

import time

# Architecture 1: SimpleToolAgent
print("\n" + "─"*80)
print("1️⃣  SIMPLETOOLAGENT")
print("─"*80)

agent1 = SimpleToolAgent(
    config=config,
    tools=[azure_search_tool],
    max_iterations=3,
    verbose=False
)

start1 = time.time()
response1 = agent1.invoke(question_benchmark)
time1 = time.time() - start1

print(f"⏱️  Temps: {time1:.2f}s")
print(f"💡 Réponse: {response1[:300]}...\n" if len(response1) > 300 else f"💡 Réponse: {response1}\n")

# Architecture 2: SimpleReActAgent
print("\n" + "─"*80)
print("2️⃣  SIMPLEREACTAGENT")
print("─"*80)

agent2 = SimpleReActAgent(
    tools=[azure_search_tool],
    max_iterations=3,
    verbose=False
)

start2 = time.time()
response2 = agent2.invoke(question_benchmark)
time2 = time.time() - start2

print(f"⏱️  Temps: {time2:.2f}s")
print(f"💡 Réponse: {response2[:300]}...\n" if len(response2) > 300 else f"💡 Réponse: {response2}\n")

# Architecture 3: HierarchicalPlanner
print("\n" + "─"*80)
print("3️⃣  HIERARCHICALPLANNER")
print("─"*80)

agent3 = HierarchicalPlanner(
    tools=[azure_search_tool],
    max_steps=5
)

start3 = time.time()
result3 = agent3.execute(question_benchmark)
time3 = time.time() - start3
response3 = result3['final_result']

print(f"⏱️  Temps: {time3:.2f}s")
print(f"📋 Plan steps: {len(result3['plan'])}")
print(f"💡 Réponse: {response3[:300]}...\n" if len(response3) > 300 else f"💡 Réponse: {response3}\n")

# Résumé
print("\n" + "="*80)
print("📊 RÉSUMÉ COMPARATIF")
print("="*80)
print(f"\nSimpleToolAgent      : {time1:.2f}s - Réponse directe et efficace")
print(f"SimpleReActAgent     : {time2:.2f}s - Raisonnement explicite")
print(f"HierarchicalPlanner  : {time3:.2f}s - Planification et décomposition")

fastest = min(time1, time2, time3)
if fastest == time1:
    print("\n🏆 SimpleToolAgent est le plus rapide pour cette question")
elif fastest == time2:
    print("\n🏆 SimpleReActAgent est le plus rapide pour cette question")
else:
    print("\n🏆 HierarchicalPlanner est le plus rapide pour cette question")

print("\n💡 La 'meilleure' architecture dépend de votre cas d'usage :")
print("   - Production & vitesse ? → SimpleToolAgent")
print("   - Compréhension & débogage ? → SimpleReActAgent")
print("   - Tâches complexes multi-étapes ? → HierarchicalPlanner")

---

# L'Outil de Génération de Rapports

## Introduction

Le **report_writer_tool** permet de générer des rapports professionnels dans différents formats. C'est un outil puissant pour transformer des informations brutes en documents structurés.

### Types de Rapports Disponibles

1. **general** : Rapport généraliste
2. **technical** : Rapport technique avec spécifications
3. **business** : Rapport d'affaires avec KPIs
4. **customer_analysis** : Analyse client avec insights

### Exemple 1 : Générer un Rapport Technique

In [ ]:
# Agent avec recherche + rapport
report_agent = SimpleToolAgent(
    config=config,
    tools=[azure_search_tool, report_writer_tool],
    system_prompt="Tu es un assistant qui crée des rapports professionnels sur VéloCorp.",
    max_iterations=5,
    verbose=False
)

# Demander un rapport technique
demande_rapport = """Crée un rapport technique sur le modèle E-City.
Inclus les caractéristiques, spécifications techniques, et mode d'emploi.
Format: Markdown"""

print("📄 GÉNÉRATION DE RAPPORT TECHNIQUE")
print("="*80)
print(f"\n📝 Demande: {demande_rapport}\n")

rapport_technique = report_agent.invoke(demande_rapport)

print("\n" + "─"*80)
print("📋 RAPPORT GÉNÉRÉ")
print("─"*80)
print(rapport_technique)

### Exemple 2 : Rapport d'Analyse Client

In [ ]:
# Demander un rapport d'analyse client
demande_analyse = """Crée un rapport d'analyse client sur les retours concernant les vélos VéloCorp.
Analyse les problèmes courants, la satisfaction, et les recommandations.
Type: customer_analysis
Format: Markdown"""

print("📊 GÉNÉRATION DE RAPPORT D'ANALYSE CLIENT")
print("="*80)
print(f"\n📝 Demande: {demande_analyse}\n")

rapport_client = report_agent.invoke(demande_analyse)

print("\n" + "─"*80)
print("📋 RAPPORT GÉNÉRÉ")
print("─"*80)
print(rapport_client)

### Exemple 3 : Rapport d'Affaires avec HierarchicalPlanner

Utilisons le HierarchicalPlanner pour créer un rapport complexe nécessitant plusieurs étapes :

In [ ]:
# Créer un planner avec les deux outils
planner_report = HierarchicalPlanner(
    tools=[azure_search_tool, report_writer_tool],
    max_steps=8
)

# Objectif complexe de rapport
objectif_rapport = """Crée un rapport d'affaires complet pour le conseil d'administration de VéloCorp.
Le rapport doit inclure :
1. Vue d'ensemble des produits et leur positionnement
2. Analyse des retours clients et réclamations
3. Points d'amélioration identifiés
4. Recommandations stratégiques

Utilise le format Markdown et le type business."""

print("💼 GÉNÉRATION DE RAPPORT D'AFFAIRES COMPLEXE")
print("="*80)
print(f"\n🎯 Objectif:\n{objectif_rapport}\n")

result_rapport = planner_report.execute(objectif_rapport)

print("\n" + "─"*80)
print("📋 PLAN D'EXÉCUTION")
print("─"*80)
for step in result_rapport['plan']:
    status = "✅" if step['status'] == 'completed' else "⏳"
    print(f"{status} Step {step['step_id']}: {step['description'][:70]}")

print("\n" + "─"*80)
print("📄 RAPPORT FINAL")
print("─"*80)
print(result_rapport['final_result'])

### 🎛️ À Vous d'Expérimenter !

Créez vos propres rapports :

In [ ]:
# À VOUS : Créez différents types de rapports

# Idées de rapports à tester :
idees_rapports = [
    "Rapport technique sur tous les modèles de vélos (type: technical)",
    "Analyse des questions fréquentes sur la garantie (type: customer_analysis)",
    "Rapport d'affaires sur la stratégie produit (type: business)",
    "Synthèse des problèmes techniques courants (type: technical)"
]

# Choisissez votre type de rapport
mon_rapport = idees_rapports[0]  # ← Changez l'index ou écrivez votre demande

print(f"📝 Génération de rapport: {mon_rapport}\n")

mon_agent_rapport = SimpleToolAgent(
    config=config,
    tools=[azure_search_tool, report_writer_tool],
    max_iterations=5,
    verbose=True  # Pour voir le processus
)

resultat_mon_rapport = mon_agent_rapport.invoke(mon_rapport)
print(f"\n{'='*80}\n")
print(resultat_mon_rapport)

---

# Exercices Pratiques

## Exercice 1 : Choisir la Bonne Architecture

Pour chaque scénario ci-dessous, quelle architecture utiliseriez-vous ?

**Scénarios :**
1. Un chatbot client doit répondre rapidement aux questions simples sur les produits
2. Un système de débogage doit expliquer pourquoi un agent a pris certaines décisions
3. Un workflow d'analyse mensuelle nécessite de collecter des données de plusieurs sources, les analyser, puis générer un rapport
4. Une API de production doit servir des réponses en moins de 2 secondes
5. Un outil pédagogique doit montrer aux étudiants comment un agent raisonne

**Vos réponses :** (à discuter)

In [ ]:
# Exercice 1 - Vos réponses
mes_choix = {
    "scenario_1": "SimpleToolAgent",  # ← Votre choix : SimpleToolAgent, SimpleReActAgent, ou HierarchicalPlanner
    "scenario_2": "",  # ← À compléter
    "scenario_3": "",  # ← À compléter
    "scenario_4": "",  # ← À compléter
    "scenario_5": ""   # ← À compléter
}

# Vérification (réponses suggérées)
reponses_suggerees = {
    "scenario_1": "SimpleToolAgent - Rapidité et fiabilité pour production",
    "scenario_2": "SimpleReActAgent - Transparence du raisonnement",
    "scenario_3": "HierarchicalPlanner - Gestion de workflow complexe multi-étapes",
    "scenario_4": "SimpleToolAgent - Performance optimale",
    "scenario_5": "SimpleReActAgent - Visibilité du processus de raisonnement"
}

print("Vos choix vs Réponses suggérées:\n")
for scenario, choix in mes_choix.items():
    print(f"{scenario}:")
    print(f"  Votre choix: {choix}")
    print(f"  Suggéré: {reponses_suggerees[scenario]}")
    print()

## Exercice 2 : Créer Votre Propre Workflow

Créez un workflow complet qui :
1. Cherche les infos sur un modèle spécifique de vélo
2. Cherche les problèmes clients associés
3. Génère un rapport de synthèse

In [ ]:
# Exercice 2 - À compléter

# Choisissez l'architecture appropriée
# Option 1: SimpleToolAgent
# Option 2: HierarchicalPlanner

# TODO: Créez votre agent
mon_agent = None  # ← À compléter

# TODO: Définissez votre question/objectif
ma_question = ""  # ← À compléter

# TODO: Invoquez l'agent
# mon_resultat = ...

# TODO: Affichez le résultat
# print(mon_resultat)

## Exercice 3 : Analyser les Différences

Prenez une question complexe et testez-la avec les 3 architectures. Analysez :
- Temps d'exécution
- Qualité de la réponse
- Transparence du processus
- Facilité de débogage si quelque chose ne va pas

In [ ]:
# Exercice 3 - Votre analyse comparative

ma_question_test = "Analyse les retours clients sur le modèle Pro-Sécurité et recommande des améliorations"

# TODO: Testez avec SimpleToolAgent
# ...

# TODO: Testez avec SimpleReActAgent
# ...

# TODO: Testez avec HierarchicalPlanner
# ...

# TODO: Comparez les résultats

---

# Conclusion

## Ce que Vous Avez Appris

✅ **Trois architectures d'agents différentes** :
- SimpleToolAgent : Efficace et direct
- SimpleReActAgent : Transparent et explicable
- HierarchicalPlanner : Stratégique et organisé

✅ **Quand utiliser chaque architecture** selon vos besoins

✅ **Comment intégrer des outils** (recherche, génération de rapports)

✅ **L'importance des paramètres** (max_iterations, max_steps, system_prompt)

✅ **La génération de rapports professionnels** avec des outils spécialisés

## Principes Clés du Monde Agentique

1. **Il n'y a pas d'architecture "meilleure"** - seulement des architectures adaptées à différents cas d'usage

2. **La transparence a un coût** - SimpleReActAgent et HierarchicalPlanner sont plus lents mais plus explicables

3. **Les outils sont essentiels** - Sans outils, les agents ne peuvent pas interagir avec le monde réel

4. **La planification aide pour la complexité** - Les tâches multi-étapes bénéficient de la décomposition

5. **L'itération a des limites** - max_iterations et max_steps empêchent les boucles infinies

## Prochaines Étapes

Vous êtes maintenant prêt à :
- Créer vos propres agents pour des cas d'usage spécifiques
- Intégrer de nouveaux outils personnalisés
- Combiner plusieurs agents dans des workflows complexes
- Explorer des architectures avancées (agents multi-agents, réflexion, etc.)

**Félicitations ! Vous comprenez maintenant le monde agentique ! 🎉**

---

## Questions de Réflexion Finale

1. **Explicabilité vs Performance** : Dans quel contexte privilégieriez-vous l'explicabilité même au prix de la performance ?

2. **Limites des Agents** : Quelles sont les limitations que vous avez observées ? Comment pourrait-on les surmonter ?

3. **Cas d'Usage Réel** : Pensez à un cas d'usage dans votre domaine. Quelle architecture choisiriez-vous et pourquoi ?

4. **Évolution Future** : Comment imaginez-vous l'évolution des systèmes agentiques dans les 2-3 prochaines années ?

5. **Éthique et Responsabilité** : Quelles questions éthiques soulèvent les agents autonomes qui prennent des décisions ?